**1. First, let's create a pricing function and check the std **

In [1]:
import numpy as np

In [27]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [28]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
# Comparing the sample mean with the real mean
(default_time.mean(), 1/def_rate)

(9.94702690956294, 10.0)

In [35]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4):
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    P=[]
    for i in default_time:
        if (i<mat):
            p_def=recovery*np.exp(-rf_rate*i)
            P.append(p_def)
        else:
            p_mat=np.exp(-rf_rate*mat)
            P.append(p_mat)
    price=np.mean(P)
    return price

# Call your function
corp_bond(mat, def_rate, rf_rate, recovery, n_sample)

0.4331710082064671

In [41]:
# Find the mean and std by calling the function 100 times. 
num_sampling=100
p_stat=[]
for i in range(num_sampling):
    p=corp_bond(mat, def_rate, rf_rate, recovery, n_sample)
    p_stat.append(p)
    
print(np.mean(p_stat),np.std(p_stat))

0.44010674316132303 0.0023379168988373724



**2. Now, let's improve the function by reducing the MC variations.**<p>
    (1)Use antithetic method: If U is uniform random variable, so is 1-U<p>
    (2)Also shift the RNs to match the mean, 1/def_rate



In [56]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=int(n_sample/2))
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))#concatenate U with 1-U to make 10000 samples

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(10.0, 10.0)

In [57]:
# Now include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):
    if(antithetic):
        U = np.random.uniform(size= int(n_sample/2))
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    else:
        U = np.random.uniform(size=n_sample)
        default_time = -(1/def_rate)*np.log(U)
        
    if(mean_match):
        default_time += 1/def_rate-default_time.mean()
        
    P=[]
    for i in default_time:
        if (i<mat):
            p_def=recovery*np.exp(-rf_rate*i)
            P.append(p_def)
        else:
            p_mat=np.exp(-rf_rate*mat)
            P.append(p_mat)
            
    price=np.mean(P)
    return price

In [59]:
corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample)

0.4418416282089862

In [60]:
# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both
num_sampling2=100
p_stat2=[]
for i in range(num_sampling):
    p=corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample)
    p_stat2.append(p)
    
print(np.mean(p_stat2),np.std(p_stat2))

0.44038699524977276 0.0015179982715957461


**3. Finally, what is the analytic value of the corporate bond?<p>How does it compare to your MC result above?**

In [65]:
r=def_rate+rf_rate
real_price=(1-np.exp(-r*mat))*0.3*def_rate/r+np.exp(-r*mat)
print('The analytic value is',real_price)
print('The MC value is',np.mean(p_stat2))
print('The spread is',real_price-np.mean(p_stat2))

The analytic value is 0.44040907156462505
The MC value is 0.44038699524977276
The spread is 2.207631485229289e-05
